In [1]:
from IPython.display import Image

In [7]:
from google.cloud import bigquery
%load_ext google.cloud.bigquery

# What are the most popular Citibike destinations for couples in NYC?

> From Tim Swast's blog: [What are the most popular Citibike destinations for NYC couples?](https://medium.com/@TimSwast/what-are-the-most-popular-citibike-destinations-for-nyc-couples-1baf646fbba)

Let’s use BigQuery and the [New York City public dataset on Citibike trips](https://cloud.google.com/bigquery/public-data/nyc-citi-bike) to find the most and *least* romantic spots to bike to with your significant other.

## The Query

Breaking this query down, we group Citibike trips together into ones that start and end at about the same time. If the grouped-together trip had more than one person in it, we call it a couple. (Obviously this could be a trip with friends and/or family, too.) Then, I find which stations have the highest percentage of trips from couples versus single-rider trips.

In [22]:
%%bigquery popularspots

#standardSql
SELECT
  group_trips / (single_trips + group_trips) AS percent_groups,
  single_trips + group_trips AS total_trips,
  q.end_station_id AS end_station_id,
  stations.name AS name,
  stations.latitude AS latitude,
  stations.longitude AS longitude
FROM (
  SELECT
    COUNTIF(group_size = 1) AS single_trips,
    COUNTIF(group_size != 1) AS group_trips,
    end_station_id
  FROM (
    SELECT
      ROUND(UNIX_SECONDS(starttime) / 120) AS start,
      -- round to nearest 2 minutes
      ROUND(UNIX_SECONDS(stoptime) / 120) AS stop,
      -- round to nearest 2 minutes
      start_station_id,
      end_station_id,
      COUNT(*) AS group_size
    FROM
      `bigquery-public-data.new_york.citibike_trips`
    GROUP BY
      start,
      stop,
      start_station_id,
      end_station_id )
  GROUP BY
    end_station_id ) q
LEFT JOIN
  `bigquery-public-data.new_york.citibike_stations` AS stations
ON
  q.end_station_id = stations.station_id
WHERE
  name is not NULL
ORDER BY
  percent_groups DESC

,percent_groups,total_trips,end_station_id,name,latitude,longitude
0,0.299927,6885,3282,5 Ave & E 88 St,40.783070,-73.959390
1,0.256076,25551,232,Cadman Plaza E & Tillary St,40.695977,-73.990149
2,0.243590,32448,3143,5 Ave & E 78 St,40.776321,-73.964274
3,0.224591,2814,3292,5 Ave & E 93 St,40.785785,-73.957481
4,0.222798,8299,3180,Brooklyn Bridge Park - Pier 2,40.698780,-73.997120
5,0.214978,3712,3374,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955613
6,0.212913,34089,3137,5 Ave & E 73 St,40.772828,-73.966853
7,0.202688,33184,3165,Central Park West & W 72 St,40.775794,-73.976206
8,0.193279,19759,3160,Central Park West & W 76 St,40.778968,-73.973747
9,0.190643,139963,2006,Central Park S & 6 Ave,40.765909,-73.976342


## The results

The number 1 spot for couples to bike to together is the 5 Ave & E 88 St Citibike station, right by Central Park, with 30% of trips completed by couples. Also in top spots were the other stations around Central Park: 
5 Ave & E 78 St and 5 Ave & E 93 St.

The runner-up is Cadman Plaza E & Tillary St in Brooklyn, with 26% of trips completed by couples. Third place goes to Brooklyn Bridge Park — Pier 2, with 22% of trips completed by couples.

Where does the data say you shouldn’t you take your significant other? Only 1% of trips completed at the Penn Station Valet station were by couples.

## Mapping it out

We can use some jupyter features and the Google Static Maps API to visualize these locations.

In [130]:
import os
from urllib.parse import urlencode

def map_with_markers(center, markers):
    markers = [f"color:{marker['color']}|{marker['latitude']},{marker['longitude']}" for marker in markers]
    params = {
        'key': os.environ.get('GOOGLE_API_KEY'),
        'size': '1200x1200',
        'center': center,
        'zoom': 12,
        'markers': markers
    }
    base_url = 'https://maps.googleapis.com/maps/api/staticmap'
    url = f"{base_url}?{urlencode(params, doseq=True)}"
    return Image(url=url, format='jpg')

import colorsys

def color_for_magnitude(mag, hue):
    r, g, b = map(lambda x: int(x * 255), colorsys.hsv_to_rgb(hue, 1.0-mag, 1.0))
    return f'0x{r:02x}{g:02x}{b:02x}'

### Map of the most popular spots

In [135]:
top_15 = popularspots.to_dict('records')[:15]
markers = [
    {'latitude': location['latitude'] , 'longitude': location['longitude'], 'color': color_for_magnitude(rank/15, 0.6)}
    for rank, location in enumerate(top_15, 1)]

map_with_markers('', markers)

### Map of the least popular spots

In [132]:
bottom_15 = popularspots.to_dict('records')[-15:]
markers = [
    {'latitude': location['latitude'] , 'longitude': location['longitude'], 'color': color_for_magnitude(rank/15, 0.0)}
    for rank, location in enumerate(bottom_15, 1)]

map_with_markers('', markers)